<a href="https://colab.research.google.com/github/le-Mon94/HateOffensiveSpeech-Detector-FromAudio/blob/main/Detector_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hate / Offensive Speech Detector from Audio with Whisper AI and sklearn LinearSVC**



## **Downloading and Importing libraries**

In [1]:
 # Installing Hugging Face Transformer and Whisper AI

!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!pip install pydub

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-doa29i7e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-doa29i7e
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798395 sha256=9701a4810e990dbb9c8769a6ef9f0c3f25231f5238d39e4d0e8cbe24e4ae68a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-t2bwsr3z/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5i6l4pyz
  Running command git clone --filter=blob:none --quiet https://github.com/open

In [2]:
# Importing Google Colab stuff, whisper model, pydub and audio display

from google.colab import files # files from colab
from google.colab import data_table # data_table to make dataframe look nicer
from IPython.display import Audio, display # Audio Display
from pydub import AudioSegment # Audio Spliter
import whisper # whisper
import os # os library
import math # math
import numpy as np

import statistics
from statistics import mode # to find mode

# whisper model

WhisperModel = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:48<00:00, 31.4MiB/s]


In [3]:
# Import LinearSVC Stuff

import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Detection Model from LinearSVC**

**Importing Dataset and spliting**

In [4]:
# Importing Dataset

url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
df = pd.read_csv(url)
del df['Unnamed: 0']

In [5]:
import random

original_offensive_df = df[df['class'] == 1]

# Get the indices of examples to be removed
num_examples_to_remove = len(original_offensive_df) - 1200
indices_to_remove = random.sample(list(original_offensive_df.index), num_examples_to_remove)

# Remove the selected examples
downsampled_offensive_df = original_offensive_df.drop(indices_to_remove)

# Update the original DataFrame with the downsampled class 1 data
df = pd.concat([df[df['class'] != 1], downsampled_offensive_df])

# Shuffle the DataFrame to ensure randomness
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
df.head(20)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,2,1,0,0,Good Mernin faggots http://t.co/3ZPTPqFkTl
1,3,0,0,3,2,#HappyColumbusDay .....hate to sound all #PETA...
2,3,2,1,0,0,RT @Kidd_BravO615: Nigga wanna bitch about a p...
3,3,0,3,0,1,Them Bad bitches blow kisses by my earlobe
4,3,0,0,3,2,RT @radiotour_en: Duchess of Cambridge to pres...
5,3,0,0,3,2,RT @PeteRose_14: On this day 1941 the #Yankees...
6,3,0,3,0,1,&#8220;@TyLawson3: F**k it!!!!! I'm never wear...
7,3,0,1,2,2,@ennuitimes clever monkey!
8,3,0,1,2,2,Lmao redskins turnin up
9,3,0,0,3,2,Hand me a brew and I'll chug it


In [7]:
class_counts = df['class'].value_counts()

print(class_counts)

2    4163
0    1430
1    1200
Name: class, dtype: int64


In [8]:
x = df['tweet']
y = df['class']

In [9]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in word_tokenize(nopunc) if word and not re.search(pattern=r"\s+", string=word)]

def split_text(text):
    return text.split()

train_splits = []

for i in tqdm_notebook(range(df.shape[0])):
    train_splits.append(' '.join(process_text(df['tweet'][i])))

  0%|          | 0/6793 [00:00<?, ?it/s]

In [10]:
x = train_splits
y = df['class']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train, x_test, y_train_encoded, y_test_encoded = train_test_split(x, y_encoded, test_size=0.35, random_state=42)

**Model Fitting and Training**

In [11]:
# Define the text classification pipeline

Model = Pipeline([
    ('vect', CountVectorizer(tokenizer=split_text, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])

# Fit the pipeline to your data

Model.fit(x_train, y_train_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function split_text at 0x7bb715b391b0>)),
                ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])

In [12]:
# Output and Training everything

label_mapping = {0: 'hate', 1: 'offensive', 2: 'neither'}

pred_labels_encoded = Model.predict(x_test)

print(x_test)

print(pred_labels_encoded)

pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

submit_df = pd.DataFrame({'text': x_test, 'predicted_label': pred_labels})

submit_df['class'] = submit_df['predicted_label'].map(label_mapping)

submit_df.to_csv('predictions.csv', index=False)

new_df = pd.read_csv('predictions.csv')
new_df.head(20)

['I just feel like my mouthpiece so cold 1041185 Ill give lessons to every niggah that ever got turned down for the low cost of 2499 1043358 Hmu', 'Natitude Okay Im only going to say it once COMB yer BEARDS lol wiggers', 'RT librarianknight More queer lady Templars Thats my point', 'RT eidolonebooks dreamt i was in a trash bag btw', 'knvcklepvck gim me one with his mouth closed I cant draw teeth without making them look retarded oMF', 'I was flying down the slope and just rammed into this small Asian boy kid got up like a champ and walked it off', '97339733973397339733 GRAND OPENING 97339733973397339733 TOP CLASS 9733 Oriental Massage 9733 PERFECT 9733 httptcoFhGqF0RWBE', 'RT TheChrisCates Look at this faggot httptcodb7SIri8qg', 'WimpWDiesel yu forgot we beat yall Thats like a garbage man calling a janitor trash', 'My favorite DC restaurant is Charlie Palmers SweetMeLissa sassylibrarian1 taotaosalupa tingtingbabyyyy', 'Hold all your ghetto labeled and endorsed vodka brands you can rum 

,text,predicted_label,class
0,I just feel like my mouthpiece so cold 1041185...,1,offensive
1,Natitude Okay Im only going to say it once COM...,2,neither
2,RT librarianknight More queer lady Templars Th...,2,neither
3,RT eidolonebooks dreamt i was in a trash bag btw,2,neither
4,knvcklepvck gim me one with his mouth closed I...,2,neither
5,I was flying down the slope and just rammed in...,2,neither
6,97339733973397339733 GRAND OPENING 97339733973...,2,neither
7,RT TheChrisCates Look at this faggot httptcodb...,0,hate
8,WimpWDiesel yu forgot we beat yall Thats like ...,2,neither
9,My favorite DC restaurant is Charlie Palmers S...,2,neither


**Classification Report**

In [13]:
y_hat = Model.predict(x_train)
report = classification_report(y_train_encoded, y_hat)

conf_matrix = confusion_matrix(y_train_encoded, y_hat)

# Compute accuracy
accuracy = accuracy_score(y_train_encoded, y_hat)

# Print the classification report and other metrics
print("Classification Report:")
print(report)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nAccuracy: {:.4f}".format(accuracy))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       940
           1       1.00      1.00      1.00       782
           2       1.00      1.00      1.00      2693

    accuracy                           1.00      4415
   macro avg       1.00      1.00      1.00      4415
weighted avg       1.00      1.00      1.00      4415


Confusion Matrix:
[[ 940    0    0]
 [   0  782    0]
 [   0    0 2693]]

Accuracy: 1.0000


In [14]:
y_hat = Model.predict(x_test)
report = classification_report(y_test_encoded, y_hat)

conf_matrix = confusion_matrix(y_test_encoded, y_hat)

# Compute accuracy
accuracy = accuracy_score(y_test_encoded, y_hat)

# Print the classification report and other metrics
print("Classification Report:")
print(report)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nAccuracy: {:.4f}".format(accuracy))

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.64      0.70       490
           1       0.76      0.70      0.73       418
           2       0.90      0.98      0.94      1470

    accuracy                           0.86      2378
   macro avg       0.82      0.77      0.79      2378
weighted avg       0.85      0.86      0.85      2378


Confusion Matrix:
[[ 313   74  103]
 [  69  294   55]
 [  18   17 1435]]

Accuracy: 0.8587


## **Audio to Text by Whisper AI**

**Audio splitting function for long audios (>1 min)**

In [15]:
# Audio splitter function (Found online, stack oveflow)


class SplitAudio():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename

        self.audio = AudioSegment.from_mp3(self.filepath)

    def get_duration(self):
        return self.audio.duration_seconds

    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="mp3")

    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                print('All splited successfully')

# Folder Clearing

def clear(path):
  for files in os.listdir(path):
    filepath = os.path.join(path, files)
    if os.path.isfile(filepath):
        os.remove(filepath)

**Uploading Audio**

In [16]:
# Creating new folder

new_folder = '/content/audio_input'
os.makedirs(new_folder, exist_ok=True)

# Folder path and changing dictionary

folder_path = new_folder
os.chdir(folder_path)

# Folder Clearing

clear(folder_path)

# Audio file uploading

upload = files.upload()

# Finding file name and path then store it in "file_path"

file_name = next(iter(upload))

file_path = folder_path + "/" + file_name

# Print to check

print(str(file_path))

# Displaying Audio

display(Audio(file_path, autoplay=True))

Saving AudioHigh.mp3 to AudioHigh.mp3
/content/audio_input/AudioHigh.mp3


**Audio Splitting**

In [17]:
# Audio splitting

split_audio = SplitAudio(folder_path, file_name)
split_audio.multiple_split(min_per_split=1)

# Removing original audio

file_to_rm = os.path.join(folder_path, file_name)
if os.path.exists(file_to_rm):
  os.remove(file_to_rm)

# Display

for p_audio in sorted(os.listdir(folder_path)):
  audio_path = os.path.join(folder_path, p_audio)
  print(audio_path)
  display(Audio(audio_path, autoplay=False))

0 Done
All splited successfully
/content/audio_input/0_AudioHigh.mp3


**Whisper AI transcribe**

In [18]:
# Whisper

Result_List = []

def sp2t(audiofile):

  text_result = WhisperModel.transcribe(audiofile)

  return text_result

for p_audio in sorted(os.listdir(folder_path)):
  audio_path = os.path.join(folder_path, p_audio)

  result = sp2t(audio_path)
  Result_List.append(result["text"])
  print(result["text"])

print("List :")
print(Result_List)

 I live in a high-income housing environment that goes by the government name of First Class. Me and a group of my allies invest stocks to order to run our successful business. We posses exotic vehicles, mansions, expensive clothing and only use debit cards for financial purchases. If anyone would like to settle alterations it'll be more than happy to give out loans. I would like you to know I am a very successful person and I regularly invest stocks.
List :
[" I live in a high-income housing environment that goes by the government name of First Class. Me and a group of my allies invest stocks to order to run our successful business. We posses exotic vehicles, mansions, expensive clothing and only use debit cards for financial purchases. If anyone would like to settle alterations it'll be more than happy to give out loans. I would like you to know I am a very successful person and I regularly invest stocks."]


## **Result / Model Prediciton**

**Predict into dataframe**

In [19]:
# Predict

data_table.enable_dataframe_formatter()

pred = Model.predict(Result_List)
pred_labels = label_encoder.inverse_transform(pred)

# Show Result as Dataframe

result_df = pd.DataFrame({'text': Result_List, 'predicted_label': pred_labels})

result_df['class'] = result_df['predicted_label'].map(label_mapping)

result_df

,text,predicted_label,class
0,I live in a high-income housing environment t...,2,neither


**Predicted List and Mode Finding**

In [20]:
# Show Result as a List and finding mode

predicted_list = []

for predictions in pred_labels:

  predicted_list.append(label_mapping[predictions])

print(predicted_list)

print("This Speech Is Mostly : ",mode(predicted_list))

['neither']
This Speech Is Mostly :  neither


**Text Input (For no audio testing)**

In [21]:
# Text Input

TextInput_List = []

while True:
  text = str(input("Input text here :"))
  TextInput_List.append(text)

  choice = str(input("Do you wish to add more Y/N :"))

  if choice == "Y":
    continue
  else:
    break

print("List : ", TextInput_List)

# Predict

data_table.enable_dataframe_formatter()

pred = Model.predict(TextInput_List)
pred_labels = label_encoder.inverse_transform(pred)

# Show Result as Dataframe

result_df = pd.DataFrame({'text': TextInput_List, 'predicted_label': pred_labels})

result_df['class'] = result_df['predicted_label'].map(label_mapping)

result_df

Input text here :You know, maybe it's not right to jaywalk in the middle of the night.
Do you wish to add more Y/N :Y
Input text here :I hate you
Do you wish to add more Y/N :n
List :  ["You know, maybe it's not right to jaywalk in the middle of the night.", 'I hate you']


,text,predicted_label,class
0,"You know, maybe it's not right to jaywalk in t...",2,neither
1,I hate you,0,hate
